In [ ]:
!pip install transformers==4.47.0 accelerate==1.1.1   ### VERY IMPORTANT, TOOK ME A LONG WHILE TO UNDERSTAND THAT IT WORKS ON THIS VERSION (on Kaggle)
#  torch==2.5.1+cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 71.0 MB/s eta 0:00:00:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 333.2/333.2 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 29.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 70.4 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.9.41
  

In [8]:
!nvidia-smi

Thu Jul 31 02:52:56 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   39C    P8              9W /   70W |       3MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
!git clone https://github.com/AlphaAnas/DSnoT.git # this repository on Github contains the implementation / fork of DSnoT finetuning method
                                                # which contains DSnoT code and SparseGPT, and Wanda pruning methods. I also added normal L1 Pruning method 
                                                # there copying from Torch Pruning repository



Cloning into 'DSnoT'...
remote: Enumerating objects: 283, done.
remote: Counting objects: 100% (189/189), done.
remote: Compressing objects: 100% (138/138), done.
remote: Total 283 (delta 91), reused 143 (delta 51), pack-reused 94 (from 1)
Receiving objects: 100% (283/283), 1.55 MiB | 10.90 MiB/s, done.
Resolving deltas: 100% (130/130), done.


In [8]:
%cd DSnoT

/kaggle/working/DSnoT


In [9]:
!ls -a

.		  .gitignore			  main.py
..		  imgs				  pruning-dsnot-procedure.ipynb
DEBUG_GUIDE.md	  lib				  __pycache__
debug_run.ps1	  llama-3b-DSnoT-results.txt	  README.md
debug_run.py	  llama-3b-l1-results.txt	  torch_pruning
environment.yaml  llama-3b-sparsegpt-results.txt
.git		  magnitude.py


In [ ]:
!huggingface-cli login --token 


### DOWNLOAD THE DATASET

In [11]:
# Step 1: Download the dataset files
!wget -q https://huggingface.co/datasets/allenai/c4/resolve/main/en/c4-train.00000-of-01024.json.gz
!wget -q https://huggingface.co/datasets/allenai/c4/resolve/main/en/c4-validation.00000-of-00008.json.gz

# Step 2: Create the 'en' directory (if it doesn't exist)
!mkdir -p en

# Step 3: Move the downloaded files into the 'en' directory
!mv c4-train.00000-of-01024.json.gz en/
!mv c4-validation.00000-of-00008.json.gz en/

# Optional Step 4: Change directory to working directory (not needed unless your code explicitly requires it)
# %cd /kaggle/working


### Downloading and Loading the dataset manually to avoid errors in the pre-written code of library

In [12]:
from datasets import load_dataset

traindata = load_dataset('json', data_files='en/c4-train.00000-of-01024.json.gz', split='train')
valdata = load_dataset('json', data_files='en/c4-validation.00000-of-00008.json.gz', split='train')  # still 'train' because there's only one split

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

### Default arguments - each method will edit this below as per its requirements

In [13]:
class Args:
    model = "meta-llama/Llama-3.2-3B-Instruct"
    model_type = "llama"  # will be inferred automatically
    seed = 0
    nsamples = 128
    eval_dataset = 'wikitext2'
    sparsity_ratio = 0.2
    sparsity_type = 'unstructured'
    prune_method = 'magnitude'
    initial_method = 'magnitude'
    max_cycle_time = 50
    without_DSnoT = True
    update_threshold = 0.1
    pow_of_var_regrowing = 1
    pow_of_var_pruning = 1  # default not overridden
    skip_layer = 'no_skip'
    skip_sub_layer = 'no_skip'
    without_same_sign = 'True'
    get_time_overhead = False
    output_results_file = 'results.txt'
    cache_dir = 'llama-3b-weights'
    save_model = "pruned-llama-3b-L2"
args = Args()

### Main function of loading the model and calling the Pruning methods

In [ ]:
import os
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from importlib.metadata import version
from datasets import load_dataset
import random

from lib.prune import check_sparsity, prune_DSnoT, prune_magnitude, prune_sparsegpt, prune_wanda
from lib.prune_opt import check_sparsity_opt, prune_DSnoT_opt
from lib.eval import eval_ppl
from lib.save_results import save_ppl_result
from lib.data import get_loaders, TokenizerWrapper # Import TokenizerWrapper



from magnitude import prune_model

print('torch', version('torch'))
print('transformers', version('transformers'))
print('accelerate', version('accelerate'))
print('# of gpus: ', torch.cuda.device_count())



def get_llm(model, cache_dir="llm_weights"):
    model = AutoModelForCausalLM.from_pretrained(
        model,
        torch_dtype=torch.float16,
        cache_dir=cache_dir,
        low_cpu_mem_usage=True,
        device_map="auto"
    )

    model.seqlen = 2048
    # model.seqlen = 128
    return model


def main(args):


        # Set random seeds
        np.random.seed(args.seed)
        torch.random.manual_seed(args.seed)

        # Determine model type
        if not args.model_type:
            if any(model_name in args.model for model_name in ["llama", "vicuna"]):
                args.model_type = "llama"
            elif "opt" in args.model:
                args.model_type = "opt"
            else:
                print("Warning: Could not determine model type from model name.")
                return
        print(f"model type: {args.model_type}")

        prune_n, prune_m = 0, 0
        if args.sparsity_type != "unstructured":
            assert args.sparsity_ratio == 0.5, "sparsity ratio must be 0.5 for structured N:M sparsity"
            prune_n, prune_m = map(int, args.sparsity_type.split(":"))

        print(f"loading llm model {args.model}")
        model = get_llm(args.model, args.cache_dir)
        model.eval()
        tokenizer = AutoTokenizer.from_pretrained(args.model, use_fast=False)
        if tokenizer.pad_token is None:
            tokenizer.pad_token = tokenizer.eos_token


            # Check initial sparsity
        initial_sparsity = check_sparsity(model)
        print(f"Initial sparsity: {initial_sparsity:.4f}")
        
        # Get initial parameter count
        initial_params = sum(p.numel() for p in model.parameters()) / 1e6
        print(f"Initial parameters: {initial_params:.2f}M")




        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    

        if "30b" in args.model or "65b" in args.model:
            device = model.hf_device_map["lm_head"]

        
        print(f"Using device: {device}")
        print(f"Model: {args.model}")
        print(f"Sparsity ratio: {args.sparsity_ratio}")
        print(f"Samples: {args.nsamples}")

        if args.sparsity_ratio != 0:
            print("pruning starts")


            # we are currently only dealing with llama
            if args.model_type == "llama":
                if args.prune_method == "wanda":
                    prune_wanda(args, model, tokenizer, device, prune_n=prune_n, prune_m=prune_m)


                # normal l1 pruning
                elif args.prune_method == "magnitude_normal":

                    prune_magnitude(args, model, tokenizer, device, prune_n=prune_n, prune_m=prune_m)

                # https://github.com/VainF/Torch-Pruning - magnitude pruning using dep-graph
                elif args.prune_method == "magnitude_torch_prune":


                    prune_model(
                        model, tokenizer,
                        pruning_type=args.prune_method,
                        weight_metric="l1", model_type=args.model_type,
                        device=device,
                        max_seq_len=2048, pruning_ratio=args.sparsity_ratio,
                        save_path=args.save_path
                            )
                    
                elif args.prune_method == "sparsegpt":
                          prune_sparsegpt(args, model, tokenizer, device, prune_n=prune_n, prune_m=prune_m, save_path=args.save_path)

                # a fine tuning method that would prune using anyof the above method (currently done using wanda)
                elif args.prune_method == "DSnoT":
                    prune_DSnoT(args, model, tokenizer, device, prune_n=prune_n, prune_m=prune_m, save_path=args.save_path)




        # I HAVEN'T USED OR IMPLEMENTED THE OPT based MODELS 
        # elif args.model_type == "opt":
        #     if args.prune_method == "wanda":
        #         prune_wanda_opt(args, model, tokenizer, device, prune_n=prune_n, prune_m=prune_m)
        #     elif args.prune_method == "magnitude":
        #         prune_magnitude_opt(args, model, tokenizer, device, prune_n=prune_n, prune_m=prune_m)
        #     elif args.prune_method == "sparsegpt":
        #         prune_sparsegpt_opt(args, model, tokenizer, device, prune_n=prune_n, prune_m=prune_m)
        #     elif args.prune_method == "DSnoT":
        #         prune_DSnoT_opt(args, model, tokenizer, device, prune_n=0, prune_m=0, save_path=args.save_path)

        print("*" * 30)
        sparsity_ratio = check_sparsity(model) if args.model_type == "llama" else check_sparsity_opt(model)
        print(f"sparsity sanity check {sparsity_ratio:.4f}")
        print("*" * 30)

    # Get final parameter count
        final_params = sum(p.numel() for p in model.parameters()) / 1e6
        print(f"Final parameters: {final_params:.2f}M")

        compression_ratio = (initial_params - final_params) / initial_params * 100
        print(f"Compression ratio: {compression_ratio:.2f}%")
        

        # dataset = 'wikitext2'
        # ppl = eval_ppl(model, tokenizer, dataset, device)
        # print(f"\nppl on {dataset}: {ppl}\n")

        # save_ppl_result(args, args.output_results_file, sparsity_ratio, ppl)

        if args.save_model:
            model.save_pretrained(args.save_model)
            tokenizer.save_pretrained(args.save_model)

torch 2.6.0+cu124
transformers 4.47.0
accelerate 1.1.1
# of gpus:  2


### Evaluate Perplxity before pruning 

In [ ]:
print("Evaluating perplexity Before Pruning")
model = get_llm(args.model, args.cache_dir)
tokenizer = AutoTokenizer.from_pretrained(args.model, use_fast=False)
dataset = 'wikitext2'
device = torch.device("cuda:0")
ppl = eval_ppl(model, tokenizer, dataset, device)
print(f"\nppl on {dataset}: {ppl}\n")

### USING Torch Pruning , Dep-Graph L1 PRUNING ON LLAMA 3B (working)

 Note: 1- (1-pruning_ratio) ^2 will be the real pruning ratio 
so if the pruning ratio = 0.3 then the model will be 0.5 % pruned (only for this dep-graph version)


In [15]:
args = Args()
args.model = "meta-llama/Llama-3.2-3B-Instruct"
args.model_type = "llama"  # will be inferred automatically
args.sparsity_ratio = 0.17 # setting 0.1 so pruning ratio becomes roughly 0.19 
args.sparsity_type = 'unstructured'
args.prune_method = 'magnitude_torch_prune' # right now it only supports L1 pruning - will add L2 as well
args.initial_method = 'magnitude_torch_prune'
args.output_results_file = 'llama-3b-l2-torch_prune-results.txt'
# args.cache_dir = r'C:\Users\hp-15\Disc D\scrapeyard\GSCP\pruning\DSNOT2\babylm-10m-weights'
# args.cache_dir = '/home/meesum/.cache/huggingface/hub'
# args.save_model = "pruned-llama-3b_torch_prune-l2"
args.cache_dir = 'llama-3b-weights'
args.save_path = "pruned-llama-3b_torch_prune-l1_20_per"


In [16]:
main(args)

model type: llama
loading llm model meta-llama/Llama-3.2-3B-Instruct


config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

2025-07-30 02:47:47.647950: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753843667.989848      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753843668.097022      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

layer 0 sparsity 0.000002
layer 1 sparsity 0.000001
layer 2 sparsity 0.000001
layer 3 sparsity 0.000001
layer 4 sparsity 0.000001
layer 5 sparsity 0.000002
layer 6 sparsity 0.000001
layer 7 sparsity 0.000001
layer 8 sparsity 0.000001
layer 9 sparsity 0.000001
layer 10 sparsity 0.000001
layer 11 sparsity 0.000002
layer 12 sparsity 0.000002
layer 13 sparsity 0.000002
layer 14 sparsity 0.000002
layer 15 sparsity 0.000001
layer 16 sparsity 0.000001
layer 17 sparsity 0.000001
layer 18 sparsity 0.000001
layer 19 sparsity 0.000001
layer 20 sparsity 0.000001
layer 21 sparsity 0.000001
layer 22 sparsity 0.000001
layer 23 sparsity 0.000001
layer 24 sparsity 0.000001
layer 25 sparsity 0.000001
layer 26 sparsity 0.000001
layer 27 sparsity 0.000001
Initial sparsity: 0.0000
Initial parameters: 3212.75M
Using device: cuda:0
Model: meta-llama/Llama-3.2-3B-Instruct
Sparsity ratio: 0.17
Samples: 128
pruning starts
Original model parameters: 3212.75M


/kaggle/working/DSnoT/torch_pruning/dependency.py:712: UserWarning: Unwrapped parameters detected: ['model.layers.19.post_attention_layernorm.weight', 'model.layers.22.post_attention_layernorm.weight', 'model.layers.7.input_layernorm.weight', 'model.layers.8.input_layernorm.weight', 'model.layers.2.input_layernorm.weight', 'model.layers.2.post_attention_layernorm.weight', 'model.layers.3.input_layernorm.weight', 'model.layers.4.post_attention_layernorm.weight', 'model.layers.11.input_layernorm.weight', 'model.layers.13.input_layernorm.weight', 'model.layers.14.input_layernorm.weight', 'model.layers.16.input_layernorm.weight', 'model.layers.18.input_layernorm.weight', 'model.layers.25.post_attention_layernorm.weight', 'model.layers.23.post_attention_layernorm.weight', 'model.layers.27.input_layernorm.weight', 'model.layers.1.input_layernorm.weight', 'model.layers.3.post_attention_layernorm.weight', 'model.layers.8.post_attention_layernorm.weight', 'model.layers.10.input_layernorm.weight

Starting pruning process...
LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 3072) => (embed_tokens): Embedding(128256, 2548)
    (layers): ModuleList(
      (0-27): 28 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=3072, out_features=3072, bias=False) => (q_proj): Linear(in_features=2548, out_features=2048, bias=False)
          (k_proj): Linear(in_features=3072, out_features=1024, bias=False) => (k_proj): Linear(in_features=2548, out_features=1024, bias=False)
          (v_proj): Linear(in_features=3072, out_features=1024, bias=False) => (v_proj): Linear(in_features=2548, out_features=1024, bias=False)
          (o_proj): Linear(in_features=3072, out_features=3072, bias=False) => (o_proj): Linear(in_features=2048, out_features=2548, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=3072, out_features=8192, bias=False

In [ ]:
!rm -rf 

### USING Normal L1/L2 PRUNING ON LLAMA 3B (working)

In [27]:
args = Args()
args.model = "meta-llama/Llama-3.2-3B-Instruct"
args.model_type = "llama"  # will be inferred automatically
args.sparsity_ratio = 0.2
args.sparsity_type = 'unstructured'
args.prune_method = 'magnitude_normal' # right now it only supports L1 pruning - will add L2 as well
args.initial_method = 'magnitude_normal'
args.output_results_file = 'llama-3b-l1-results.txt'
args.cache_dir = 'llama-3b-weights'
args.save_model = "pruned-llama-3b-l1"


In [28]:
main(args)

model type: llama
loading llm model meta-llama/Llama-3.2-3B-Instruct


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

use device  cuda:0
pruning starts
******************************
layer 0 sparsity 0.200704
layer 1 sparsity 0.200664
layer 2 sparsity 0.200563
layer 3 sparsity 0.200524
layer 4 sparsity 0.200458
layer 5 sparsity 0.200772
layer 6 sparsity 0.200737
layer 7 sparsity 0.200478
layer 8 sparsity 0.200906
layer 9 sparsity 0.200651
layer 10 sparsity 0.200559
layer 11 sparsity 0.200568
layer 12 sparsity 0.200702
layer 13 sparsity 0.200883
layer 14 sparsity 0.200824
layer 15 sparsity 0.200509
layer 16 sparsity 0.200653
layer 17 sparsity 0.200361
layer 18 sparsity 0.200644
layer 19 sparsity 0.200709
layer 20 sparsity 0.200461
layer 21 sparsity 0.201008
layer 22 sparsity 0.200360
layer 23 sparsity 0.200439
layer 24 sparsity 0.200517
layer 25 sparsity 0.200541
layer 26 sparsity 0.200574
layer 27 sparsity 0.200991
sparsity sanity check 0.2006
******************************


In [13]:
print(f"Allocated Memory of GPU: {torch.cuda.memory_allocated() / 1024**2:.2f} MB")
print(f"Cached Memory of GPU: {torch.cuda.memory_reserved() / 1024**2:.2f} MB")

Allocated Memory of GPU: 0.00 MB
Cached Memory of GPU: 0.00 MB


In [14]:
import torch
import gc

# Delete all relevant variables
# del model  # Replace with your model variable name
# del inputs, outputs  # Replace with any other tensors or variables
gc.collect()  # Run Python garbage collector
torch.cuda.empty_cache()  # Clear PyTorch's GPU cache

In [12]:
print(f"Allocated Memory of GPU: {torch.cuda.memory_allocated() / 1024**2:.2f} MB")
print(f"Cached Memory of GPU: {torch.cuda.memory_reserved() / 1024**2:.2f} MB")

Allocated Memory of GPU: 0.00 MB
Cached Memory of GPU: 0.00 MB


### USING DSnoT PRUNING ON LLAMA 3B (working)

In [ ]:
args = Args()
args.model = "meta-llama/Llama-3.2-3B-Instruct"
args.model_type = "llama"  # will be inferred automatically
args.sparsity_ratio = 0.2
args.nsamples = 2
args.sparsity_type = 'unstructured'
args.prune_method = 'DSnoT' #choices=["wanda", "sparsegpt", "magnitude", "DSnoT"]
args.initial_method = 'sparsegpt' # options = [magnitude, wanda, sparsegpt]
args.max_cycle_time =50
args.update_threshold=0.1
args.pow_of_var_regrowing=1
args.output_results_file = 'llama-3b-DSnoT-results.txt'
args.cache_dir = 'llama-3b-weights'
args.save_model = "pruned_DSnoT_llama_3b"
args.seed = 0
args.max_cycle_time = 10
args.update_threshold = 0.1
args.pow_of_var_regrowing = 1
args.pow_of_var_pruning = 1
args.skip_layer = "no_skip"
args.skip_sub_layer = "no_skip"
args.without_DSnoT = False
args.without_same_sign = "True"
args.get_time_overhead = False
args.output_results_file = "test_results.txt"
args.save_path= "pruned_llama3b_instruct_dsnot_with_sparsegpt"






In [25]:
main(args)

model type: llama
loading llm model meta-llama/Llama-3.2-3B-Instruct


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

layer 0 sparsity 0.000002
layer 1 sparsity 0.000001
layer 2 sparsity 0.000001
layer 3 sparsity 0.000001
layer 4 sparsity 0.000001
layer 5 sparsity 0.000002
layer 6 sparsity 0.000001
layer 7 sparsity 0.000001
layer 8 sparsity 0.000001
layer 9 sparsity 0.000001
layer 10 sparsity 0.000001
layer 11 sparsity 0.000002
layer 12 sparsity 0.000002
layer 13 sparsity 0.000002
layer 14 sparsity 0.000002
layer 15 sparsity 0.000001
layer 16 sparsity 0.000001
layer 17 sparsity 0.000001
layer 18 sparsity 0.000001
layer 19 sparsity 0.000001
layer 20 sparsity 0.000001
layer 21 sparsity 0.000001
layer 22 sparsity 0.000001
layer 23 sparsity 0.000001
layer 24 sparsity 0.000001
layer 25 sparsity 0.000001
layer 26 sparsity 0.000001
layer 27 sparsity 0.000001
Initial sparsity: 0.0000
Initial parameters: 3212.75M
Using device: cuda:0
Model: meta-llama/Llama-3.2-3B-Instruct
Sparsity ratio: 0.2
Samples: 2
pruning starts
Starting DSNOT PRUNING....
Original model parameters: 3212.75M
loading calibdation data
[INFO

### USING WANDA BASED PRUNING ON LLAMA 3B (working)

In [ ]:
args = Args()
args.model = "meta-llama/Llama-3.2-3B-Instruct"
args.model_type = "llama"  # will be inferred automatically
args.sparsity_ratio = 0.2
args.nsamples = 2
args.sparsity_type = 'unstructured'
args.prune_method = 'wanda' #choices=["wanda", "sparsegpt", "magnitude", "DSnoT"]
args.max_cycle_time =50
args.update_threshold=0.1
args.pow_of_var_regrowing=1
args.output_results_file = 'llama-3b-wanda-results.txt'
args.cache_dir = 'llama-3b-weights'
args.save_model = "pruned_wanda_llama_3b"
args.seed = 0
args.max_cycle_time = 10
args.update_threshold = 0.1
args.pow_of_var_regrowing = 1
args.pow_of_var_pruning = 1
args.skip_layer = "no_skip"
args.skip_sub_layer = "no_skip"
args.without_DSnoT = True
args.without_same_sign = "True"
args.get_time_overhead = False
args.output_results_file = "test_results.txt"
args.save_path="SAVE_PATH_FOR_YOUR_MODEL_HERE"

In [14]:
main(args)

model type: llama
loading llm model meta-llama/Llama-3.2-3B-Instruct


config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

2025-07-26 17:24:55.561159: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753550695.764210      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753550695.817618      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

layer 0 sparsity 0.000002
layer 1 sparsity 0.000001
layer 2 sparsity 0.000001
layer 3 sparsity 0.000001
layer 4 sparsity 0.000001
layer 5 sparsity 0.000002
layer 6 sparsity 0.000001
layer 7 sparsity 0.000001
layer 8 sparsity 0.000001
layer 9 sparsity 0.000001
layer 10 sparsity 0.000001
layer 11 sparsity 0.000002
layer 12 sparsity 0.000002
layer 13 sparsity 0.000002
layer 14 sparsity 0.000002
layer 15 sparsity 0.000001
layer 16 sparsity 0.000001
layer 17 sparsity 0.000001
layer 18 sparsity 0.000001
layer 19 sparsity 0.000001
layer 20 sparsity 0.000001
layer 21 sparsity 0.000001
layer 22 sparsity 0.000001
layer 23 sparsity 0.000001
layer 24 sparsity 0.000001
layer 25 sparsity 0.000001
layer 26 sparsity 0.000001
layer 27 sparsity 0.000001
Initial sparsity: 0.0000
Initial parameters: 3212.75M
Using device: cuda:0
Model: meta-llama/Llama-3.2-3B-Instruct
Sparsity ratio: 0.2
Samples: 2
pruning starts
loading calibdation data
[INFO] Loading training and validation datasets...
[INFO] Datasets l

The attention layers in this model are transitioning from computing the RoPE embeddings internally through `position_ids` (2D tensor with the indexes of the tokens), to using externally computed `position_embeddings` (Tuple of tensors, containing cos and sin). In v4.46 `position_ids` will be removed and `position_embeddings` will be mandatory.


pruning layer 0 name self_attn.q_proj
pruning layer 0 name self_attn.k_proj
pruning layer 0 name self_attn.v_proj
pruning layer 0 name self_attn.o_proj
pruning layer 0 name mlp.gate_proj
pruning layer 0 name mlp.up_proj
pruning layer 0 name mlp.down_proj
pruning layer 1 name self_attn.q_proj
pruning layer 1 name self_attn.k_proj
pruning layer 1 name self_attn.v_proj
pruning layer 1 name self_attn.o_proj
pruning layer 1 name mlp.gate_proj
pruning layer 1 name mlp.up_proj
pruning layer 1 name mlp.down_proj
pruning layer 2 name self_attn.q_proj
pruning layer 2 name self_attn.k_proj
pruning layer 2 name self_attn.v_proj
pruning layer 2 name self_attn.o_proj
pruning layer 2 name mlp.gate_proj
pruning layer 2 name mlp.up_proj
pruning layer 2 name mlp.down_proj
pruning layer 3 name self_attn.q_proj
pruning layer 3 name self_attn.k_proj
pruning layer 3 name self_attn.v_proj
pruning layer 3 name self_attn.o_proj
pruning layer 3 name mlp.gate_proj
pruning layer 3 name mlp.up_proj
pruning layer 3

### USING SparseGPT PRUNING ON LLAMA 3B (working)

In [ ]:
args = Args()
args.model = "meta-llama/Llama-3.2-3B-Instruct"
args.model_type = "llama"  # will be inferred automatically
args.sparsity_ratio = 0.2
args.nsamples = 2
args.sparsity_type = 'unstructured'
args.prune_method = 'sparsegpt' #choices=["wanda", "sparsegpt", "magnitude", "DSnoT"]
args.max_cycle_time =50
args.update_threshold=0.1
args.pow_of_var_regrowing=1
args.output_results_file = 'llama-3b-sparsegpt-results.txt'
args.cache_dir = 'llama-3b-weights'
args.save_model = "pruned_sparsegpt_llama_3b"
args.seed = 0
args.max_cycle_time = 10
args.update_threshold = 0.1
args.pow_of_var_regrowing = 1
args.pow_of_var_pruning = 1
args.skip_layer = "no_skip"
args.skip_sub_layer = "no_skip"
args.without_same_sign = "True"
args.get_time_overhead = False
args.output_results_file = "test_results.txt"
args.save_path = "pruned-llama-3b_instruct_sparsegpt"

In [17]:
main(args)

model type: llama
loading llm model meta-llama/Llama-3.2-3B-Instruct


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

layer 0 sparsity 0.000002
layer 1 sparsity 0.000001
layer 2 sparsity 0.000001
layer 3 sparsity 0.000001
layer 4 sparsity 0.000001
layer 5 sparsity 0.000002
layer 6 sparsity 0.000001
layer 7 sparsity 0.000001
layer 8 sparsity 0.000001
layer 9 sparsity 0.000001
layer 10 sparsity 0.000001
layer 11 sparsity 0.000002
layer 12 sparsity 0.000002
layer 13 sparsity 0.000002
layer 14 sparsity 0.000002
layer 15 sparsity 0.000001
layer 16 sparsity 0.000001
layer 17 sparsity 0.000001
layer 18 sparsity 0.000001
layer 19 sparsity 0.000001
layer 20 sparsity 0.000001
layer 21 sparsity 0.000001
layer 22 sparsity 0.000001
layer 23 sparsity 0.000001
layer 24 sparsity 0.000001
layer 25 sparsity 0.000001
layer 26 sparsity 0.000001
layer 27 sparsity 0.000001
Initial sparsity: 0.0000
Initial parameters: 3212.75M
Using device: cuda:0
Model: meta-llama/Llama-3.2-3B-Instruct
Sparsity ratio: 0.2
Samples: 2
pruning starts
Starting pruning process...
Original model parameters: 3212.75M
Loading dataset...
[INFO] Loa

The attention layers in this model are transitioning from computing the RoPE embeddings internally through `position_ids` (2D tensor with the indexes of the tokens), to using externally computed `position_embeddings` (Tuple of tensors, containing cos and sin). In v4.46 `position_ids` will be removed and `position_embeddings` will be mandatory.


Catcher forward pass: sample 0
Catcher forward pass: sample 1
Data fed. Removing Catcher.
Cache collected. Starting pruning.

--- Pruning Layer 0 ---
Finding prunable sublayers...
Found 7 sublayers to prune.
Initialized SparseGPT for self_attn.q_proj
Initialized SparseGPT for self_attn.k_proj
Initialized SparseGPT for self_attn.v_proj
Initialized SparseGPT for self_attn.o_proj
Initialized SparseGPT for mlp.gate_proj
Initialized SparseGPT for mlp.up_proj
Initialized SparseGPT for mlp.down_proj
Registering forward hook for self_attn.q_proj
Registering forward hook for self_attn.k_proj
Registering forward hook for self_attn.v_proj
Registering forward hook for self_attn.o_proj
Registering forward hook for mlp.gate_proj
Registering forward hook for mlp.up_proj
Registering forward hook for mlp.down_proj
Running forward passes for calibration...
Layer 0, Sample 0
Layer 0, Sample 1
Removing forward hooks.
Pruning sublayer self_attn.q_proj in layer 0
Pruned self_attn.q_proj in 1.26s
Pruning sub

### A function for measuing the number of weights / parameters zeroed out of the model 

In [ ]:
import os
import torch
from transformers import AutoModelForCausalLM

def load_model_stats(model_path):
    model = AutoModelForCausalLM.from_pretrained(model_path, local_files_only=True, torch_dtype=torch.float16)  # or float32
    model.to("cpu")  # or "cuda" if your GPU supports it


    total_params = sum(p.numel() for p in model.parameters()) / 1e6
    nonzero_params = sum((p != 0).sum().item() for p in model.parameters() if p.requires_grad) /1e6

    # size_gb = sum(os.path.getsize(os.path.join(dp, f)) for dp, _, fs in os.walk(model_path) for f in fs) / 1e9

    print(f"Total Parameters      : {total_params:.2f} million")
    print(f"Non-zero Parameters   : {nonzero_params:.2f} million")
    print(f"Sparsity              : {(1 - nonzero_params / total_params) * 100:.2f}%")

    return model


In [ ]:
load_model_stats("/kaggle/working/DSnoT/pruned_llama3b_instruct_dsnot_with_sparsegpt") ## give the path where the pruned model is saved

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Total Parameters      : 3212.75 million
Non-zero Parameters   : 2649.34 million
Sparsity              : 17.54%


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 3072)
    (layers): ModuleList(
      (0-27): 28 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=3072, out_features=3072, bias=False)
          (k_proj): Linear(in_features=3072, out_features=1024, bias=False)
          (v_proj): Linear(in_features=3072, out_features=1024, bias=False)
          (o_proj): Linear(in_features=3072, out_features=3072, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=3072, out_features=8192, bias=False)
          (up_proj): Linear(in_features=3072, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=3072, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
      )
    )
    (norm

In [ ]:
### ALL METHODS OF PRUNING DONE

In [28]:
import torch
import gc

# Delete all relevant variables
# del model  # Replace with your model variable name
# del inputs, outputs  # Replace with any other tensors or variables
gc.collect()  # Run Python garbage collector
torch.cuda.empty_cache()  # Clear PyTorch's GPU cacheimport torch

import gc

# Clear PyTorch's CUDA cache
if torch.cuda.is_available():
    torch.cuda.empty_cache()

# Optional: Also clear Python's garbage collector
gc.collect()

0

In [29]:
print(f"Allocated Memory of GPU: {torch.cuda.memory_allocated() / 1024**2:.2f} MB")
print(f"Cached Memory of GPU: {torch.cuda.memory_reserved() / 1024**2:.2f} MB")

Allocated Memory of GPU: 8.12 MB
Cached Memory of GPU: 20.00 MB


### Function to find the folder size of the pruned model to see the size diff b/w original and pruned model

In [30]:
import os

def get_folder_size(folder_path):
    total_size = 0
    for dirpath, dirnames, filenames in os.walk(folder_path):
        for filename in filenames:
            file_path = os.path.join(dirpath, filename)
            try:
                total_size += os.path.getsize(file_path)
            except OSError as e:
                print(f"Error getting size of {file_path}: {e}")
    return total_size



In [26]:
# Path of your Original model to see if it is pruned and now smaller than original 
folder_path = "/kaggle/working/DSnoT/llama-3b-weights"

if os.path.exists(folder_path):
    size_in_bytes = get_folder_size(folder_path)
    size_in_kb = size_in_bytes / 1024
    size_in_mb = size_in_kb / 1024
    print("SIZE OF ORIGINAL MODEL IS : ")
    print(f"Folder size: {size_in_bytes:,} bytes")
    print(f"Folder size: {size_in_kb:.2f} KB")
    print(f"Folder size: {size_in_mb:.2f} MB")
else:
    print(f"Folder not found: {folder_path}")

SIZE OF ORIGINAL MODEL IS : 
Folder size: 12,851,102,108 bytes
Folder size: 12549904.40 KB
Folder size: 12255.77 MB


In [31]:
# Path of your pruned model to see if it is pruned and now smaller than original 
folder_path = "/kaggle/working/DSnoT/pruned_llama3b_instruct_dsnot_with_sparsegpt"

if os.path.exists(folder_path):
    size_in_bytes = get_folder_size(folder_path)
    size_in_kb = size_in_bytes / 1024
    size_in_mb = size_in_kb / 1024
    print("SIZE OF PRUNED MODEL IS : ")
    print(f"Folder size: {size_in_bytes:,} bytes")
    print(f"Folder size: {size_in_kb:.2f} KB")
    print(f"Folder size: {size_in_mb:.2f} MB")
else:
    print(f"Folder not found: {folder_path}")

SIZE OF PRUNED MODEL IS : 
Folder size: 6,442,815,753 bytes
Folder size: 6291812.26 KB
Folder size: 6144.35 MB


In [ ]:
print(f"Allocated Memory of GPU: {torch.cuda.memory_allocated() / 1024**2:.2f} MB")
print(f"Cached Memory of GPU: {torch.cuda.memory_reserved() / 1024**2:.2f} MB")

### UPLOAD THE MODEL TO HF

In [17]:

# Create repo
repo_id = "AlphaAnas70/pruned-llama-3b_torch_prune-l1_20_per"
create_repo(repo_id, exist_ok=True)

# Upload model folder
upload_folder(
    repo_id=repo_id,
    folder_path="/kaggle/working/DSnoT/pruned-llama-3b_torch_prune-l1_20_per",
    commit_message="Pushing sparsegpt pruned LLaMA model"
)


Uploading...:   0%|          | 0.00/5.11G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/AlphaAnas70/pruned-llama-3b_torch_prune-l1_20_per/commit/9bafe4d91c07eaff5059051a2cda963d10334145', commit_message='Pushing sparsegpt pruned LLaMA model', commit_description='', oid='9bafe4d91c07eaff5059051a2cda963d10334145', pr_url=None, repo_url=RepoUrl('https://huggingface.co/AlphaAnas70/pruned-llama-3b_torch_prune-l1_20_per', endpoint='https://huggingface.co', repo_type='model', repo_id='AlphaAnas70/pruned-llama-3b_torch_prune-l1_20_per'), pr_revision=None, pr_num=None)

### Load to see if the model is now working 

In [9]:
from transformers import AutoModelForCausalLM, AutoTokenizer

repo_id = "AlphaAnas70/pruned_llama-3.2-3b-Instruct-20_per"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(repo_id,  use_fast=False)

# Load model
model = AutoModelForCausalLM.from_pretrained(repo_id)



RuntimeError: Failed to import transformers.models.auto.modeling_auto because of the following error (look up to see its traceback):
Failed to import transformers.generation.utils because of the following error (look up to see its traceback):
cannot import name 'check_fp8_capability' from 'accelerate.utils.environment' (/usr/local/lib/python3.11/dist-packages/accelerate/utils/environment.py)

In [ ]:
input_text = "Once upon a time"
inputs = tokenizer(input_text, return_tensors="pt")

# Generate output
outputs = model.generate(**inputs, max_new_tokens=50)

# Decode and print
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


## RANDOMLY CHECKING JUST A MODEL THAT IS PRUNED ALR

In [2]:
from transformers import AutoModelForCausalLM
import torch

model = AutoModelForCausalLM.from_pretrained(
    "AlphaAnas70/pruned_llama3b_sparsegpt_17.55_per",
    torch_dtype=torch.float16,
    device_map="cpu"
)

total_params = sum(p.numel() for p in model.parameters()) / 1e6
nonzero_params = sum((p != 0).sum().item() for p in model.parameters()) / 1e6
sparsity = (1 - nonzero_params / total_params) * 100

print(f"Total params: {total_params:.2f}M")
print(f"Non-zero params: {nonzero_params:.2f}M")
print(f"Sparsity: {sparsity:.2f}%")


config.json:   0%|          | 0.00/927 [00:00<?, ?B/s]

2025-08-12 16:33:31.629847: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755016411.970168      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755016412.066731      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

Total params: 3212.75M
Non-zero params: 2649.03M
Sparsity: 17.55%
